In [1]:
import numpy as np
import k3d

In [2]:
inch = 25.4 # mm

In [3]:
class Point:
    def __init__(self, x, y, z=0):
        self.data = [float(x) * inch, float(y) * inch, float(z) * inch]

    def __getitem__(self, idx):
        return self.data[idx]

    def __array__(self):
        return np.array(self.data)

    def __repr__(self):
        return f"{self.data}"

In [4]:
np.array(Point(1,2,3))

array([25.4, 50.8, 76.2])

In [5]:
class Line:
    def __init__(self, a, *b, **arg):
        self.data = [a] + [i for i in b]
        try:
            self.closed = arg["closed"]
        except KeyError:
            self.closed = True

    def __getitem__(self, idx):
        return self.data[idx]

    def __iter__(self):
        if self.closed:
            return iter(self.data + [self.data[0]])
        else:
            return iter(self.data)

    def __array__(self):
        return np.array([np.array(i) for i in self])

    def __repr__(self):
        return f"{list(self)} closed:{self.closed}"

In [6]:
board = Line(Point(0,0),Point(2,0),Point(2,1),Point(0,1),closed=True); np.array(board)

array([[ 0. ,  0. ,  0. ],
       [50.8,  0. ,  0. ],
       [50.8, 25.4,  0. ],
       [ 0. , 25.4,  0. ],
       [ 0. ,  0. ,  0. ]])

In [7]:
scan = [[a[0], a[1], a[2] + b] for a, b in zip(board, [-1, -3, +1, +3])]
scan

[[0.0, 0.0, -1.0], [50.8, 0.0, -3.0], [50.8, 25.4, 1.0], [0.0, 25.4, 3.0]]

# corr

In [8]:
def plot(grid):
    p = k3d.plot(height=300,menu_visibility=False,background_color=0x222222,grid_color=0x224444,camera_mode='orbit')
    
    p += k3d.line(np.array(board),color=0x0000FF,width=.3)
    p += k3d.points(scan,color=0xAA0000)
    p += k3d.points(grid,color=0x00AA00)
    
    p.display()

In [9]:
x0 = board[0][0] ; y0 = board[0][1]
x1 = board[2][0] ; y1 = board[2][1]

In [37]:
# https://matandlife.blogspot.com/2015/03/maxima.html
eq = 'linsolve([\n'
for i in range(len(scan)):
    x, y, z = scan[i]
    comma = " ," if i + 1 < len(scan) else ""
    eq += f"    {z} = b1 + b2 * {x} + b3 * {y} + b4 * {x} * {y}{comma}\n"
eq += '],[b1,b2,b3,b4]);\n'
print(eq)

linsolve([
    -1.0 = b1 + b2 * 0.0 + b3 * 0.0 + b4 * 0.0 * 0.0 ,
    -3.0 = b1 + b2 * 50.8 + b3 * 0.0 + b4 * 50.8 * 0.0 ,
    1.0 = b1 + b2 * 50.8 + b3 * 25.4 + b4 * 50.8 * 25.4 ,
    3.0 = b1 + b2 * 0.0 + b3 * 25.4 + b4 * 0.0 * 25.4
],[b1,b2,b3,b4]);



In [38]:
b1 = -1.0
b2 = -5./127
b3 = 20./127
b4 = 0

In [39]:
grid = []
for x in np.linspace(x0, x1, 11):
    for y in np.linspace(y0, y1, 11):
        z = b1 + b2 * x + b3 * y + b4 * x * y
        grid += [[x, y, z]]
plot(grid)

Output()